In [9]:
from utils.mapping2 import create_dataset, create_embedding_matrix,  load_all_features
import os
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [10]:
path = os.getcwd()
print(path)

C:\Users\Abhinav\PycharmProjects\Video_Captioning


In [11]:
train_feature_src = os.path.join(path,'dataset', 'features-small-train')
test_feature_src = os.path.join(path,'dataset', 'features-small-test')
train_data_src = os.path.join(path, 'dataset', 'MSVD_train-small.csv')
test_data_src = os.path.join(path, 'dataset', 'MSVD_test-small.csv')


In [12]:
train_features, padded, target_padded, tokenizer, max_len = create_dataset(
    train_feature_src, train_data_src)

test_features, padded, target_padded, tokenizer, max_len = create_dataset(
    test_feature_src, test_data_src)

Creating Dataset...: 100%|██████████| 3177/3177 [00:01<00:00, 1835.48it/s]


In [13]:
train_features.shape, padded.shape, target_padded.shape, max_len

((3177, 20, 4096), (3177, 33), (3177, 33), 33)

In [14]:
test_feature, test_vid_ids = load_all_features(test_feature_src)
test_feature.shape, len(test_vid_ids)


Loading Features...: 100%|██████████| 20/20 [00:00<00:00, 2506.38it/s]


((20, 20, 4096), 20)

In [17]:
vocab_size = len(tokenizer.word_index)
embd_vec_size = 100
glove_src = os.path.join(path,'dataset','glove.6B.100d.txt')

In [18]:
embd_matrix = create_embedding_matrix(glove_src, tokenizer.word_index, vocab_size, embd_vec_size)

Loading Word Embeddings....: 100%|██████████| 400001/400001 [00:29<00:00, 13726.03it/s]


400000
Embed not found for 151 words


In [22]:
print(embd_matrix[10])
embd_matrix.shape, vocab_size

[ 0.59368   0.44825   0.5932    0.074134  0.11141   1.2793    0.16656
  0.2407    0.39045   0.32766  -0.75034   0.35007   0.76057   0.38067
  0.17517   0.031791  0.46849  -0.21653  -0.46282   0.39967   0.16623
 -0.011477  0.044059  0.30325   0.6153    0.47047  -0.44036  -1.5963
  0.18433   0.23193   0.20452   0.51617   0.65734  -0.3452    0.23446
 -0.62004  -0.68741   0.28575   1.0605    0.46916  -0.85149   0.10154
  0.21426  -0.20587   0.23636   0.21321  -0.21287   0.12107   0.18766
 -0.23282  -0.25499  -0.39631   0.84379   1.6801   -0.40941  -1.9976
 -0.69868   0.21732   1.2197    0.55126   0.44095   0.72588  -0.092053
 -0.022406  0.72039   0.1076    0.84116   0.30312  -0.42544   0.056362
  0.13109  -0.071181 -0.10579   0.56677   0.54547   0.84113   0.14861
 -0.62628  -0.68391  -1.0831   -0.088385  0.32167   0.47794   0.091868
 -1.2559   -1.2268    0.085401  0.36833   0.081566 -0.76611   0.87751
 -0.22008   0.82401  -0.092207 -0.45941   0.46571  -0.56018  -0.54648
  0.15162  -0.30754

((2140, 100), 2139)

In [33]:
lstm_units = 200
encoder_input_size = train_features.shape[2]
time_steps_enc = train_features.shape[1]

In [34]:
# Encoder

enc_input = Input((None, encoder_input_size), name='enc_input')
enc_lstm = LSTM(lstm_units, return_state=True, name='enc_lstm')
X = enc_input
X = enc_lstm(X)
enc_out, enc_h, enc_c = X
enc_states = [enc_h, enc_c]


In [35]:
# Decoder

dec_input = Input((None,), name='dec_input')
dec_embedding = Embedding(vocab_size+1, embd_vec_size, input_length=max_len,
                          trainable=False, name='dec_embd', mask_zero=True)
dec_lstm = LSTM(lstm_units, return_sequences=True,
                return_state=True, name='dec_lstm')
dec_dense = Dense(vocab_size+1, activation='softmax')
Y = dec_input
Y = dec_embedding(Y)
Y = dec_lstm(Y, initial_state=enc_states)
dec_out, _, _ = Y
Y = dec_dense(dec_out)

In [36]:
# Model

model = Model(inputs=[enc_input, dec_input], outputs=Y)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [37]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_input (InputLayer)          [(None, None, 4096)] 0                                            
__________________________________________________________________________________________________
dec_embd (Embedding)            (None, None, 100)    214000      dec_input[0][0]                  
__________________________________________________________________________________________________
enc_lstm (LSTM)                 [(None, 200), (None, 3437600     enc_input[0][0]                  
____________________________________________________________________________________________

In [38]:
plot_model(model, show_shapes=True)


Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [69]:
batch_size = 1024
epochs = 100

model.fit(x=[train_features,padded], y=target_padded, batch_size=batch_size, epochs=epochs)

Train on 3177 samples
Epoch 1/100
3177/3177 [==============================] - 3s 911us/sample - loss: 0.6235
Epoch 2/100
3177/3177 [==============================] - 3s 843us/sample - loss: 0.6220
Epoch 3/100
3177/3177 [==============================] - 3s 842us/sample - loss: 0.6206
Epoch 4/100
3177/3177 [==============================] - 3s 824us/sample - loss: 0.6199

In [73]:
encoder_model_inf = Model(enc_input, enc_states)

dec_inf_input_h = Input((lstm_units,))
dec_inf_input_c = Input((lstm_units,))
dec_inf_input_states = [dec_inf_input_h, dec_inf_input_c]

dec_inf_out, dec_inf_h, dec_inf_c = dec_lstm(dec_embedding(dec_input),
                                             initial_state=dec_inf_input_states)
dec_inf_states = [dec_inf_h, dec_inf_c]
dec_inf_output = dec_dense(dec_inf_out)

decoder_model_inf = Model(inputs=[dec_input] + dec_inf_input_states,
                          outputs=[dec_inf_output] + dec_inf_states)

In [74]:
encoder_model_inf.summary()
plot_model(encoder_model_inf, show_shapes=True)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_input (InputLayer)       [(None, None, 4096)]      0         
_________________________________________________________________
enc_lstm (LSTM)              [(None, 200), (None, 200) 3437600   
Total params: 3,437,600
Trainable params: 3,437,600
Non-trainable params: 0
_________________________________________________________________
Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [75]:
decoder_model_inf.summary()
plot_model(decoder_model_inf, show_shapes=True)

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_embd (Embedding)            (None, None, 100)    214000      dec_input[0][0]                  
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 200)]        0                                            
____________________________________________________________________________________________

In [76]:
word_index = tokenizer.word_index
reverse_word_index = dict([(v,k) for k,v in word_index.items()])

In [77]:
def generate(input_seq):

    states_val = encoder_model_inf.predict(input_seq)
    target_seq = [tokenizer.word_index['\t']]

    predicted_sent = []
    stop_condition = False

    while not stop_condition:

        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(
            x=[np.array(target_seq)] + states_val)
        max_val_index = np.argmax(decoder_out[0, 0])

        if reverse_word_index[max_val_index] == '\n' or max_val_index == 0 or len(predicted_sent) > max_len:
            stop_condition = True

        target_seq = [max_val_index]
        predicted_sent.append(reverse_word_index[max_val_index])
        states_val = [decoder_h, decoder_c]

    return " ".join(predicted_sent)

In [79]:
index=600
generate(np.array([train_features[index]])), tokenizer.sequences_to_texts([padded[index]])

('cat is licking its \n', ['\t cat is licking its hands \n'])

In [81]:
model.save("model_v1.h5")
encoder_model_inf.save("encoder_model_inf_v1.h5")
decoder_model_inf.save("decoder_model_inf_v1.h5")